# Query Translation
> 사용자의 입력 질문을 번역하여 검색 성능을 향상시키는 것
- 사용자 쿼리는 모호할 수 있으며, 쿼리가 잘못 작성된 경우 쿼리와 문서 간의 의미적 유사성을 기반으로 검색을 수행하기 때문에 적절한 문서를 검색할 수 없습니다.

## Approaches
- Query Re-writing

    > 쿼리를 다른 관점에서 다시 작성하는 방식
    - Multi-Query: 여러 관점에서 질문을 작성하고, 각각에 대해 검색을 수행하여, 신뢰성을 높이는 방법
    - RAG-Fusion
- Sub Question

    > 질문을 덜 추상적으로 만들기 위해 하위 질문으로 나누는 방법
    - least to most (Google)

- Step-back Question

    > 질문을 더 추상적으로 만드는 방법

<div style="text-align: center;">
    <img src="./img/transform_questions.png" alt="Indexing" style="width: 80%;">
</div>

## Multi-Query

- 멀티 쿼리의 작동 방식
1. 질문 재작성: 입력된 질문을 여러 다른 방식으로 다시 작성합니다. 예를 들어, "강아지 돌보는 방법"이라는 질문을 "개를 돌보는 방법", "강아지 케어 방법" 등으로 재작성할 수 있습니다.
2. 각 질문에 대한 검색: 재작성된 각 질문을 사용하여 독립적으로 문서를 검색합니다.
3. 결과 통합: 각 질문에 대한 검색 결과를 통합하여 최종 결과를 생성합니다. 이 과정에서 중복된 문서는 제거됩니다.

<div style="text-align: center;">
    <img src="./img/multi_query1.png" alt="Embedding" style="width: 28%; display: inline-block; margin: 0 2%;">
    <img src="./img/multi_query2.png" alt="Indexing" style="width: 63%; display: inline-block; margin: 0 2%;">
</div>

### Environment

In [2]:
%%capture
# packages
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

### Index

In [4]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


### Multi-Query

#### `질문 재작성`

In [5]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

#### `각 질문에 대한 검색`

In [6]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


6

#### `결과 통합 및 생성`

In [7]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM agents involves breaking down large tasks into smaller and more manageable subgoals. This allows the agent to efficiently handle complex tasks by dividing them into smaller steps that can be tackled individually.'

## RAG-Fusion
 - RAG Fusion은 멀티 쿼리와 매우 유사하지만, 검색된 문서에 대해 'Reciprocal Rank Fusion'이라는 순위 매김 단계를 추가로 적용하는 차이가 있습니다.

 